# Week 4 - Programming question 1
>Consider again the data set in *week 4 assignment*, and recall that we have transformed the data into classification and regression sets.  

## Classfication using GDA
Your task is to use Gaussian Discriminant Analysis (GDA) to build a classification model. To complete this assignment, make sure you:
* a) Write your own code to implement the GDA algorithm. (Do not use built-in classification functions.)
* b) Clearly explain how the GDA model works and why it can be used for classification, in particular this data set.
* c) Train your model on the given dataset and report its accuracy. Be explicit about how you measure performance (e.g., accuracy on a test set, cross-validation, etc.).
* d) Plot the decision boundary of your model and include the visualization in your report.

In [3]:
import re, math, numpy as np
import pandas as pd, matplotlib.pyplot as plt
from pathlib import Path
df_cls = pd.read_csv("classification_dataset.csv")